## BERT Tutorial

Reference: [Multi Class Text Classification With Deep Learning Using BERT](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613)  

What used?
  * BertTokenizer
  * BertForSequenceClassification
 
BertForSequenceClassification have embedding & Linear classifier. This code may fine-tune classifier part. You can change epochs and batch_size etc. depends on your computing environment.  

If you want to save model parameters, please modify line 80 in 'Running model' block. Submission file will automatically generated by end of this code.

### Requirement
  * transformers == 4.6.0
  * pytorch == 1.8.0
  * numpy == 1.1.92
  * pandas == 1.2.3
  * tqdm == 4.60.0
  * scikit-learn == 0.24.1

<a id="Table"></a>
### Table

  - [Import library](#Import)
  - [Check train set](#Trainset)
  - [Split train/validation set](#Split)
  - [Tokenizer, Encoding data](#Encode)
  - [Pre-trained model, Data loaders](#Model)
    - Can modify batch_size in this block
  - [Optimizer & Scheduler](#Optim)
    - Can modify epochs, lr, etc. in this block
  - [Define metrics](#Metrics)
  - [Running model](#Run)
  - [Test & Submission](#Sub)

<a id="Import"></a>
### Import library
  - [Return to table](#Table)

In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

ModuleNotFoundError: No module named 'transformers'

In [2]:
USE_CUDA = torch.cuda.is_available()
print('Use', 'cuda' if USE_CUDA else 'cpu')
device = torch.device('cuda' if USE_CUDA else 'cpu')

Use cuda


<a id="Trainset"></a>
### Check train set
  - [Return to table](#Table)

In [3]:
train_df = pd.read_csv('./data/train_final.csv')

Number of dataset, label distribution

In [4]:
train_df.tail()

,Id,Category,Sentence
11539,11539,3,"Although Frailty fits into a classic genre , i..."
11540,11540,1,Mediocre fable from Burkina Faso .
11541,11541,4,Like all great films about a life you never kn...
11542,11542,4,Those who are n't put off by the film 's auste...
11543,11543,4,"An ambitious movie that , like Shiner 's organ..."


In [5]:
train_df['Category'].value_counts()

3    3109
1    3015
2    2210
4    1734
0    1476
Name: Category, dtype: int64

<a id="Split"></a>
### Split train/validation set
  - [Return to table](#Table)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train_df.index.values,
                                                  train_df.Category.values,
                                                  test_size=0.01, 
                                                  random_state=42, 
                                                  stratify=train_df.Category.values)

train_df.loc[X_train, 'data_type'] = 'train'
train_df.loc[X_val, 'data_type'] = 'val'

train_df.groupby(['Category', 'data_type']).count()

Id  Sentence
Category data_type                
0        train      1461      1461
         val          15        15
1        train      2985      2985
         val          30        30
2        train      2188      2188
         val          22        22
3        train      3078      3078
         val          31        31
4        train      1716      1716
         val          18        18

In [7]:
possible_labels = train_df.Category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    
label_dict

{3: 0, 2: 1, 4: 2, 1: 3, 0: 4}

<a id="Encode"></a>
### Tokenizer, Encoding data
  - [Return to table](#Table)

In [8]:
#Use below tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

#Encode train sentence
encoded_data_train = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='train'].Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

#Encode validation sentence
encoded_data_val = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='val'].Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    #truncation=True, 
    return_tensors='pt'
)

In [9]:
"""
encoded_data_train has 3 keys:
  *input_ids
  *token_type_ids
  *attention_mask
"""

#Input
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df[train_df.data_type=='train'].Category.values)

#Validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train_df[train_df.data_type=='val'].Category.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

<a id="Model"></a>
### Pre-trained model, Data loaders
  - [Return to table](#Table)

In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=5,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [11]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

<a id="Optim"></a>
### Optimizer & Scheduler
  - [Return to table](#Table)

In [12]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

<a id="Metrics"></a>
### Define metrics
  - [Return to table](#Table)

In [13]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

<a id="Run"></a>
### Running model
  - [Return to table](#Table)

In [14]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals



    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #If you want to save model parameters, please modify below code.
    #Line 80 save model for every epochs
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    accuracy_per_class(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Epoch 1:  18%|█████████▎                                         | 65/358 [00:11<00:44,  6.56it/s, training_loss=0.453]


Epoch 1:  37%|██████████████████▎                               | 131/358 [00:21<00:35,  6.33it/s, training_loss=0.456]


Epoch 1:  55%|███████████████████████████▌                      | 197/358 [00:31<00:25,  6.38it/s, training_loss=0.432]


Epoch 1:  73%|████████████████████████████████████▋             | 263/358 [00:42<00:14,  6.42it/s, training_loss=0.409]


Epoch 1:  92%|█████████████████████████████████████████████▉    | 329/358 [00:52<00:04,  6.46it/s, training_loss=0.446]


...

Epoch 5:  55%|███████████████████████████▋                      | 198/358 [00:31<00:24,  6.43it/s, training_loss=0.207]


Epoch 5:  74%|████████████████████████████████████▊             | 264/358 [00:41<00:14,  6.48it/s, training_loss=0.181]


Epoch 5:  92%|██████████████████████████████████████████████    | 330/358 [00:51<00:04,  6.37it/s, training_loss=0.268]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:40<00:00, 56.10s/it]


Epoch 5
Training loss: 0.701007805057078
Class: 3
Accuracy: 14/15

Class: 2
Accuracy: 23/30

Class: 4
Accuracy: 8/22

Class: 1
Accuracy: 18/31

Class: 0
Accuracy: 11/18

Validation loss: 0.8747006803750992
F1 Score (Weighted): 0.6308649955486184


<a id="Sub"></a>
### Test & Submission
  - [Return to table](#Table)

In [15]:
test_df = pd.read_csv('./data/eval_final_open.csv')
test_df["Category"] = np.zeros(test_df.shape[0])

In [16]:
#Encode test sentence
encoded_data_test = tokenizer.batch_encode_plus(
    test_df.Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

dataloader_test = DataLoader(dataset_val, 
                             batch_size=batch_size)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_df.Category.values, dtype=int)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test,
                             batch_size=batch_size)

In [17]:
_, predictions, true_vals = evaluate(dataloader_test)

pred = np.argmax(predictions, axis=1)
np.unique(pred, return_counts = True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([ 532, 1138,  691, 1223,  727], dtype=int64))

In [18]:
sub = pd.read_csv("./data/sample_sub.csv")
sub['Category'] = pred

sub.to_csv("./data/210518_3.csv", index=False)